In [4]:
import numpy as np
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
import sympy
from radius_of_convergence import *

In [19]:
for i in range(10):
    print(f"\n=== Analysis for function {i+1} ===")
    analyze_function(*generate_random_polynomial(max_degree=4))


=== Analysis for function 1 ===
Generated function: -0.754365835539293*x**3 - 4.79818108606296*x**2 - 0.818767765059534*x - 0.593461185282498
Real roots: [-6.20608653355153]
Radius of convergence data: [(None, None)]

=== Analysis for function 2 ===
Generated function: -0.226136818638782*x**4 + 12.877286556852*x**3 + 4.64069164957803*x**2 + 3.78166273915329*x + 3.95891058277401
Real roots: [-0.641852923319719, 57.3079487459002]
Radius of convergence data: [(None, 42.94967), (0.0, None)]

=== Analysis for function 3 ===
Generated function: 5.91797254727108*x**2 - 7.65972946287501*x + 0.482736316351744
Real roots: [0.0664323532425581, 1.22788413803740]
Radius of convergence data: [(None, 0.64716), (0.64716, None)]

=== Analysis for function 4 ===
Generated function: 7.66174200552399*x**2 + 0.338560404095595*x + 8.73462357423302
Real roots: []
No real roots found, skipping convergence analysis.

=== Analysis for function 5 ===
Generated function: -4.81355151811204*x - 7.59155974007486
Re

In [ ]:
def generate_random_combined(trig=False, max_degree=4, phase_shift=False):
    """
    Generates a random function as a linear combination of polynomial terms (x^1 to x^max_degree) and 
    trigonometric functions (sin(x), cos(x)).

    Parameters:
    trig (bool): Whether to include trigonometric functions (sin and cos) in the basis.
    max_degree (int): The maximum degree of the polynomial terms.
    phase_shift (bool): Whether to apply a phase shift to the sine and cosine terms.

    Returns:
    tuple: A tuple containing the generated function and its symbolic expression.
    """
    # degree choice
    degree = np.random.choice(jnp.arange(max_degree), p=jnp.ones(max_degree) / max_degree)
    
    # polynomial basis
    basis = [1] + [x**d for d in range(1, degree + 2)]
    
    # trig parts
    if trig:
        coefs = jnp.round(np.random.normal(0, 5, 4), 2)
        beta = jnp.round(np.random.uniform(-2, 2), 2)
        phase = 0
        if phase_shift:
            phase = np.random.uniform(0, 2 * jnp.pi)
        
        # coefficients inside the trig
        if np.random.rand() < 0.5:
            basis.append(coefs[0] * sympy.sin(coefs[1] * x + phase))
        if np.random.rand() < 0.5:
            basis.append(coefs[2] * sympy.cos(coefs[3] * x))
    
    # coefficients outside
    coefficients = np.random.normal(0, 5, len(basis))
    
    # Create the random expression by summing the basis functions with their coefficients
    random_expr = sum(c * b for c, b in zip(coefficients, basis))
    
    # Convert the symbolic expression to a JAX-compatible function
    func = lambda x_val: jnp.array(sympy.lambdify(x, random_expr, 'numpy')(x_val))
    
    # If trigonometric terms were added, compute the period of the function
    period = None
    if trig:
        # Compute the period of the sine and cosine terms
        gcd = sympy.gcd(int(coefs[1] * 100), int(coefs[3] * 100))
        p = coefs[1] * 100 / gcd
        q = coefs[3] * 100 / gcd
        period = abs((q / coefs[1]) * 2 * jnp.pi)
    
    return func, random_expr, period

In [ ]:


plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_vals, label=str(random_expr))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Generated Random Function')
plt.legend()
plt.grid(True)
plt.show()